In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '../') # import modules from ../

# Aggregation for Statistical Output

NB: Aggregation could also be used to improve the performance of weekly plots

In [2]:
import logging
import time
import pandas as pd
import numpy as np
import datahandling as dh
from report import sensor_stats, get_week_range, get_month_range

In [3]:
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

In [4]:
import glob
input_datafiles = glob.glob('/Users/sam/Downloads/BuildAX/Millenium(bax19)/out000[0-3][0-9].bin')

In [5]:
df, dfs, t_start, t_end = dh.read_data(input_datafiles)

INFO:datahandling:Reading data from /Users/sam/Downloads/BuildAX/Millenium(bax19)/out00004.bin
INFO:datahandling:Reading data from /Users/sam/Downloads/BuildAX/Millenium(bax19)/out00010.bin
INFO:datahandling:Reading data from /Users/sam/Downloads/BuildAX/Millenium(bax19)/out00002.bin
INFO:datahandling:Reading data from /Users/sam/Downloads/BuildAX/Millenium(bax19)/out00000.bin
INFO:datahandling:Reading data from /Users/sam/Downloads/BuildAX/Millenium(bax19)/out00006.bin
INFO:datahandling:Reading data from /Users/sam/Downloads/BuildAX/Millenium(bax19)/out00008.bin
INFO:datahandling:Reading data from /Users/sam/Downloads/BuildAX/Millenium(bax19)/out00012.bin
INFO:datahandling:Reading data from /Users/sam/Downloads/BuildAX/Millenium(bax19)/out00014.bin
INFO:datahandling:Detected MIME: application/octet-stream
INFO:datahandling:Detected MIME: application/octet-stream
INFO:datahandling:Detected MIME: application/octet-stream
INFO:datahandling:Detected MIME: application/octet-stream
INFO:dat

INFO:datahandling:+ Data fixes applied in 28.18s


In [6]:
dfs = sensor_stats(dfs, 100)

INFO:report: ID      | Packets 
INFO:report:=========|=========
INFO:report:42F1D589 | 161483
INFO:report:42C6E09D | 156771
INFO:report:42362DD4 | 154219
INFO:report:42AD5F61 | 154756
INFO:report:4245B50A | 161951
INFO:report:4293D096 | 167823
INFO:report:42FF727D | 149506
INFO:report:422D4B48 | 160323
INFO:report:42C7645F | 95306
INFO:report:428A5448 | 163804
INFO:report:429284E5 | 99438
INFO:report:427B836E | 140868
INFO:report:42600766 | 154289
INFO:report:4236492B | 163100
INFO:report:421B6A82 | 1010


In [7]:
t_start, t_end

(Timestamp('2016-11-24 13:54:50'), Timestamp('2017-01-19 23:38:53'))

### Get week range

In [8]:
weeks = get_week_range(df)
len(weeks)

9

### Get month range

In [9]:
months = get_month_range(df)
len(months)

3

### Check data

In [10]:
dfs[list(dfs.keys())[0]][:5]

,Name,RSSI,Type,SequenceNo,TransmitPower,Battery,Humidity,Temp,Light,PIRCount,PIREnergy,Switch,Event,PIRDiff
DateTime,,,,,,,,,,,,,,
2016-11-24 13:54:50,42F1D589,-66,1,125,20,2767,36.84,17.6,53,0,39790,1,NaN,NaN
2016-11-24 13:55:19,42F1D589,-66,1,126,20,2767,36.84,17.7,53,0,40094,1,NaN,NaN
2016-11-24 13:55:48,42F1D589,-65,1,127,20,2767,36.84,17.7,52,0,40416,1,NaN,0.620690
2016-11-24 13:56:17,42F1D589,-64,1,128,20,2767,36.80,17.7,52,0,40720,1,NaN,-0.620690
2016-11-24 13:56:46,42F1D589,-64,1,129,20,2767,36.80,17.7,52,0,41029,1,NaN,0.172414


In [11]:
data = dfs[list(dfs.keys())[0]]

## Single Column Aggregation

In [12]:
# Define the aggregation procedure outside of the groupby operation
data.groupby(pd.TimeGrouper(freq='D'))['Temp'].agg(['mean']).rename(columns={'mean':'Temp'})[-5:]

,Temp
DateTime,
2017-01-15,11.618037
2017-01-16,15.130741
2017-01-17,16.009578
2017-01-18,16.394056
2017-01-19,16.268819


## Multi-column aggregation

In [13]:
aggregate={
    'Temp': 'mean', 
    'Humidity':'mean',
    'Light':'mean',
    'Battery': 'min'
}

data.groupby(pd.TimeGrouper(freq='12h')) \
    .agg(aggregate)[:5]


,Temp,Humidity,Light,Battery
DateTime,,,,
2016-11-24 12:00:00,15.679630,38.631200,10.289855,2767
2016-11-25 00:00:00,13.761024,38.767911,17.209556,2767
2016-11-25 12:00:00,15.523995,36.116980,18.245399,2767
2016-11-26 00:00:00,12.127040,35.311279,12.096819,2767
2016-11-26 12:00:00,12.366826,44.125089,10.350889,2767


## List of desired statistics

* Table with total monthly aggregates (for each month):
  * Warmest, Coldest (mean, min, max) ✅
  * Dryest, wettest sensor (..)       ✅
  * Brightest, darkest avg sensor (..)✅
* Range - difference between min and max, also min/max average
* 9-5 (working hours) aggregates ✅
* Average mean of all sensors (day, week, month)
* Sensors w/ best / worst signal strength
* Sensors requiring battery replacement (~2.2V)



In [14]:
df = pd.concat(dfs.values())

In [15]:
# Just values during working hours
work_hrs = ('09:00', '17:00')
df = df.iloc[df.index.indexer_between_time(work_hrs[0], work_hrs[1], include_start=True, include_end=True)]

In [32]:
def find_range(x): return np.ptp(x)

agg = df.groupby([pd.Grouper(freq='M'), 'Name'])\
    .agg({
        'Temp': ['mean', 'min', 'max', find_range],
        'Humidity': ['mean', 'min', 'max', find_range],
        'Light': ['mean', 'min', 'max'],
        'Battery': ['min']
    }).rename(columns={'find_range': 'range'})
agg

Temp                     Humidity                \
                          mean   min   max range       mean    min    max   
DateTime   Name                                                             
2016-11-30 422D4B48  15.770636  11.9  19.1   7.2  38.270863  32.88  51.09   
           42362DD4  14.453760  10.7  17.4   6.7  40.288197  35.17  52.80   
           4236492B  16.826499  13.7  19.9   6.2  36.876405  32.38  46.46   
           4245B50A  15.365600  11.2  19.5   8.3  38.178244  31.37  53.22   
           42600766  15.828280  11.0  19.8   8.8  36.882666  30.40  51.66   
           427B836E  16.512959  12.6  20.2   7.6  37.261240  31.41  50.44   
           428A5448  15.762971  12.6  18.8   6.2  37.863845  32.73  50.20   
           429284E5  15.792468  12.8  19.8   7.0  36.837157  30.87  48.51   
           4293D096  13.093906   9.4  18.0   8.6  44.239471  35.49  57.53   
           42AD5F61  16.108370  13.0  19.1   6.1  37.980283  33.10  49.89   
           42C6E09D  14.388010   9.3  18.1   8.8  40.689356  33.35  60.63   
           42C7645F  14.574767  10.9  18.9   8.0  39.245268  31.37  52.12   
           42F1D589  15.370822  11.9  18.5   6.6  38.836628  33.86  51.22   
           42FF727D  15.399612  11.0  19.1   8.1  39.915650  33.86  53.53   
2016-12-31 421B6A82  14.602083  13.9  16.3   2.4  45.386771  34.35  55.10   
           422D4B48  15.302454   8.3  21.5  13.2  44.268320  32.34  64.47   
           42362DD4  14.357034   7.6  19.6  12.0  46.482034  35.10  64.47   
           4236492B  16.144107   9.7  22.1  12.4  43.391560  32.65  68.17   
           4245B50A  14.961872   7.9  21.8  13.9  44.334961  30.88  63.28   
           42600766  15.196773   7.6  21.9  14.3  42.492748  30.50  61.85   
           427B836E  15.943657   9.1  22.5  13.4  43.621921  32.27  61.78   
           428A5448  15.567535   9.2  21.0  11.8  43.324865  31.42  62.05   
           429284E5  15.513349   9.0  22.4  13.4  42.388678  30.65  61.07   
           4293D096  13.190163   6.3  19.7  13.4  50.633249  37.57  69.03   
           42AD5F61  15.848932   9.6  21.3  11.7  43.527032  32.19  62.52   
           42C6E09D  14.076310   6.7  20.5  13.8  47.715884  31.88  70.01   
           42C7645F  14.038968   7.8  21.5  13.7  44.971500  30.96  62.94   
           42F1D589  15.199322   8.8  20.8  12.0  43.890933  33.11  61.07   
           42FF727D  14.865467   7.5  21.9  14.4  46.331923  33.27  65.49   
2017-01-31 421B6A82  14.465385  13.5  14.6   1.1  33.443462  33.00  38.36   
           422D4B48  15.551818   9.5  20.4  10.9  40.202556  30.29  58.61   
           42362DD4  13.983775   8.8  18.1   9.3  43.320699  31.49  61.68   
           4236492B  16.168022  10.5  20.0   9.5  40.563288  29.73  70.43   
           4245B50A  14.998063   9.0  19.6  10.6  40.308010  29.08  60.63   
           42600766  15.554932   8.7  20.5  11.8  38.363848  29.07  58.21   
           427B836E  15.878776  10.1  20.8  10.7  40.062629  29.28  58.43   
           428A5448  15.603158  10.3  19.7   9.4  39.837027  28.53  57.66   
           429284E5  16.111243  10.1  20.5  10.4  38.482976  28.09  56.65   
           4293D096  12.665730   7.5  16.8   9.3  47.376248  34.62  66.53   
           42AD5F61  15.869699  10.5  20.0   9.5  40.071902  29.73  57.75   
           42C6E09D  14.622535   7.7  18.6  10.9  42.469275  30.19  66.74   
           42C7645F  15.301857   8.9  19.8  10.9  39.559229  28.65  59.80   
           42F1D589  15.302523   9.8  19.1   9.3  40.320017  30.63  56.42   
           42FF727D  15.131979   8.7  19.6  10.9  41.903139  30.93  61.77   

                                 Light           Battery  
                     range        mean min   max     min  
DateTime   Name                                           
2016-11-30 422D4B48  18.21  104.422283   0   200    2784  
           42362DD4  17.63  499.137852   0  1016    2795  
           4236492B  14.08   31.151301   0    64    2780  
           4245B50A  21.85   68.824864   0   127    

In [17]:
ops = [
    ("Temp", "mean", "idxmax", "Warmest average"),
    ("Temp", "max",  "idxmax", "Warmest overall"),
    ("Temp", "mean", "idxmin", "Coldest average"),
    ("Temp", "min",  "idxmin", "Coldest overall"),
    ("Temp", "range",  "idxmin", "Largest difference"),
    
    ("Humidity", "mean", "idxmax", "Humidest average"),
    ("Humidity", "max",  "idxmax", "Humidest overall"),
    ("Humidity", "mean", "idxmin", "Dryest average"),
    ("Humidity", "min",  "idxmin", "Dryest overall"),
    
    ("Light", "mean", "idxmax", "Brightest average"),
    ("Light", "max",  "idxmax", "Brightest overall"),
    ("Light", "mean", "idxmin", "Darkest average"),
    ("Light", "min",  "idxmin", "Darkest overall")
]

In [18]:
def extract_stats(agg, ops):
    results={}
    for month in agg.index.levels[0]:
        #log.info("{0:%B %Y}".format(month))
        subframe = agg.loc[month]
        
        stats={}
        for op in ops:
            series, agg_val, operation, label = op

            name = getattr(subframe[(series, agg_val)], operation)()
            value = subframe.loc[name][(series, agg_val)]

            #log.info("{0}, {1}, {2} , {3:.1f}".format(series, label, name, value))
            if series not in stats:
                stats[series] = []
                
            stats[series].append((label, name, value))
            
        #log.info("\n")
        results[month] = stats
    return results

In [19]:
from flask_table import Table, Col, create_table

def get_table(headers, items, name):
    ItemTable = create_table(name+'Table')
    for h in headers:
        ItemTable.add_column(h, Col(h))

    # Populate, construct & return the table
    return ItemTable(items)

In [20]:
def transpose(l): 
    return list(map(list, zip(*l)))

In [21]:
stats = extract_stats(agg, ops)

In [23]:
month = list(stats.keys())[0]
container = {}
for series in stats[month]:
    internal = stats[list(stats.keys())[0]][series]
    internal = transpose(internal)
    headers = internal[0]
    internal = [dict(zip(headers, v)) for v in internal[1:]]

    table = get_table(headers, internal, series)
    container[series] = table


In [24]:
import IPython.core.display as ipd

#NestedTableCol
ipd.HTML(table.__html__())

Brightest average,Brightest overall,Darkest average,Darkest overall
42362DD4,42362DD4,42C6E09D,422D4B48
499.137851708,1016.0,30.8431064572,0.0


# Tabulate the data
First, a basic example to figure out the workings of flask_table:

In [25]:
from flask_table import Table, Col, NestedTableCol

# Dynamically create nested table using flask_table
SubItemTable = create_table('SubItemTable')\
    .add_column('col1', Col('Sub-column 1'))\
    .add_column('col2', Col('Sub-column 2'))

ItemTable = create_table('ItemTable')\
    .add_column('colA', Col('Top-column A'))\
    .add_column('subtable1', NestedTableCol('Sub Table 1 Title', SubItemTable))\
    .add_column('subtable2', NestedTableCol('Sub Table 2 Title', SubItemTable))

items = [{
    'colA':"hi",
    'subtable1':[
         {'col1': 'r2sr1c1', 'col2': 'r2sr1c2'},
         {'col1': 'r2sr2c1', 'col2': 'r2sr2c2'}
    ],
    'subtable2':[
         {'col1': 'r2sr1c1', 'col2': 'r2sr1c2'},
         {'col1': 'r2sr2c1', 'col2': 'r2sr2c2'}
    ]
}]

table = ItemTable(items)

# or {{ table }} in jinja
ipd.HTML(table.__html__())


In [26]:
from flask_table import NestedTableCol

TopTable = create_table('TopTable', base=Table)
for series in container:
    TopTable.add_column(series.lower()+'_table', NestedTableCol(series, container[series].__class__))   

items = [{series.lower()+'_table': container[series].items for series in container}]

table = TopTable(items)

# or {{ table }} in jinja
ipd.HTML(table.__html__())